In [184]:
"""
Agentic Lesson Generator using Ollama

This module implements a multi-agent system for generating high-quality educational lessons.
The system uses specialized agents for different aspects of lesson creation, orchestrated
by a central controller that manages the workflow and ensures quality.

This implementation uses Ollama for local LLM inference without requiring API keys.
"""

import os
import json
import hashlib
import time
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

# For vector database and embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# For LLM integration
from langchain_community.llms import Ollama
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# For PDF processing
from pypdf import PdfReader

# For logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Constants
DEFAULT_MODEL = "llama3"  # or "mistral" or "phi3" depending on what you have pulled
FASTER_MODEL = "llama3"  # Use the same model for consistency
EMBEDDING_MODEL = "nomic-embed-text"  # Alternative: "all-minilm"
DEFAULT_TEMPERATURE = 0.2
DEFAULT_MAX_TOKENS = 4000
OLLAMA_BASE_URL = "http://localhost:11434"  # Change if running on a different port

In [185]:
@dataclass
class LessonMetadata:
    """Metadata for a lesson, including curriculum alignment and pedagogical information."""
    subject: str
    grade_level: str
    topic: str
    subtopics: List[str]
    learning_objectives: List[str] = field(default_factory=list)
    standards_alignment: List[str] = field(default_factory=list)
    difficulty_level: str = "intermediate"
    estimated_duration: str = "45 minutes"
    prerequisites: List[str] = field(default_factory=list)
    target_skills: List[str] = field(default_factory=list)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert metadata to dictionary."""
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'LessonMetadata':
        """Create metadata from dictionary."""
        return cls(**data)


In [186]:
@dataclass
class LessonComponent:
    """A component of a lesson, such as an introduction, explanation, or activity."""
    component_type: str
    content: str
    order: int
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert component to dictionary."""
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'LessonComponent':
        """Create component from dictionary."""
        return cls(**data)


In [187]:


@dataclass
class LessonComponent:
    """A component of a lesson, such as an introduction, explanation, or activity."""
    component_type: str
    content: str
    order: int
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert component to dictionary."""
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'LessonComponent':
        """Create component from dictionary."""
        return cls(**data)

@dataclass
class Lesson:
    """A complete lesson with metadata and components."""
    title: str
    metadata: LessonMetadata
    components: List[LessonComponent] = field(default_factory=list)
    version: str = "1.0"
    created_at: float = field(default_factory=time.time)
    quality_score: Optional[float] = None
    feedback: List[str] = field(default_factory=list)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert lesson to dictionary."""
        return {
            "title": self.title,
            "metadata": self.metadata.to_dict(),
            "components": [c.to_dict() for c in self.components],
            "version": self.version,
            "created_at": self.created_at,
            "quality_score": self.quality_score,
            "feedback": self.feedback
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Lesson':
        """Create lesson from dictionary."""
        metadata = LessonMetadata.from_dict(data["metadata"])
        components = [LessonComponent.from_dict(c) for c in data["components"]]
        return cls(
            title=data["title"],
            metadata=metadata,
            components=components,
            version=data.get("version", "1.0"),
            created_at=data.get("created_at", time.time()),
            quality_score=data.get("quality_score"),
            feedback=data.get("feedback", [])
        )
    
    def to_markdown(self) -> str:
        """Convert lesson to markdown format."""
        md = f"# {self.title}\n\n"
        
        # Add metadata section
        md += "## Lesson Metadata\n\n"
        md += f"- **Subject:** {self.metadata.subject}\n"
        md += f"- **Grade Level:** {self.metadata.grade_level}\n"
        md += f"- **Topic:** {self.metadata.topic}\n"
        md += f"- **Subtopics:** {', '.join(self.metadata.subtopics)}\n"
        md += f"- **Difficulty Level:** {self.metadata.difficulty_level}\n"
        md += f"- **Estimated Duration:** {self.metadata.estimated_duration}\n\n"
        
        # Add learning objectives
        md += "### Learning Objectives\n\n"
        for i, objective in enumerate(self.metadata.learning_objectives, 1):
            md += f"{i}. {objective}\n"
        md += "\n"
        
        # Add standards alignment if available
        if self.metadata.standards_alignment:
            md += "### Standards Alignment\n\n"
            for standard in self.metadata.standards_alignment:
                md += f"- {standard}\n"
            md += "\n"
        
        # Add prerequisites if available
        if self.metadata.prerequisites:
            md += "### Prerequisites\n\n"
            for prereq in self.metadata.prerequisites:
                md += f"- {prereq}\n"
            md += "\n"
        
        # Sort components by order and add them
        sorted_components = sorted(self.components, key=lambda x: x.order)
        for component in sorted_components:
            md += f"## {component.component_type}\n\n"
            md += f"{component.content}\n\n"
        
        return md
    
    def to_json(self) -> str:
        """Convert lesson to JSON string."""
        return json.dumps(self.to_dict(), indent=2)
    
    @classmethod
    def from_json(cls, json_str: str) -> 'Lesson':
        """Create lesson from JSON string."""
        data = json.loads(json_str)
        return cls.from_dict(data)


In [188]:
class Agent:
    """Base class for all agents in the system."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = DEFAULT_TEMPERATURE):
        """Initialize the agent with a language model."""
        self.llm = Ollama(
            model=model_name,
            temperature=temperature,
            base_url=OLLAMA_BASE_URL
        )
        self.system_prompt = "You are a helpful AI assistant."
        self.name = "BaseAgent"
    
    def _call_llm(self, messages: List[Dict[str, str]]) -> str:
        """Call the language model with the given messages."""
        # Format the messages for Ollama
        formatted_prompt = ""
        for msg in messages:
            if msg["role"] == "system":
                formatted_prompt += f"<s>[INST] <<SYS>>\n{msg['content']}\n<</SYS>>\n\n"
            elif msg["role"] == "user":
                if formatted_prompt:
                    formatted_prompt += f"{msg['content']} [/INST]"
                else:
                    formatted_prompt += f"[INST] {msg['content']} [/INST]"
            elif msg["role"] == "assistant":
                formatted_prompt += f" {msg['content']} </s><s>"
        
        # Call the LLM
        response = self.llm.invoke(formatted_prompt)
        return response
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data and return output. To be implemented by subclasses."""
        raise NotImplementedError("Subclasses must implement this method")


In [189]:

class CurriculumExpertAgent(Agent):
    """Agent specialized in curriculum analysis and learning objective definition."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.1):
        """Initialize the curriculum expert agent."""
        super().__init__(model_name, temperature)
        self.name = "CurriculumExpert"
        self.system_prompt = """You are an expert curriculum designer with specialized knowledge of CBSE (Central Board of Secondary Education) Class 10 educational standards and learning objectives. Your role is to:

1. Analyze CBSE Class 10 curriculum requirements for specific subjects and topics as per NCERT guidelines
2. Define clear, measurable learning objectives aligned with CBSE assessment patterns and board exam requirements
3. Identify key concepts and skills that students should master according to CBSE syllabus and competency framework
4. Ensure alignment with CBSE educational standards, learning outcomes, and board examination patterns
5. Determine appropriate scope and sequence for lessons following NCERT textbook structure and CBSE time allocation

Focus on CBSE Class 10 core subjects including Mathematics, Science (Physics, Chemistry, Biology), Social Science (History, Geography, Political Science, Economics), English, and Hindi. Provide detailed, specific outputs that align with CBSE board exam preparation and Indian educational context.
"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to generate curriculum analysis and learning objectives."""
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        subtopics = input_data.get("subtopics","")
        grade_level = input_data.get("grade_level", "")
        special_requirements = input_data.get("special_requirements", "")
        context = input_data.get("context", "")
        
        prompt = f"""
I need a comprehensive curriculum analysis for a lesson on {topic} with sub_topics {subtopics} in {subject} for {grade_level} students.

Special requirements: {special_requirements}

Context from curriculum materials:
{context}

Please provide analysis specifically aligned with CBSE Class 10 standards:

1. A list of 3-5 specific learning objectives for this lesson (ensure they are measurable using Bloom's taxonomy and aligned with CBSE learning outcomes)
2. Key concepts from NCERT textbook that must be covered in this lesson
3. Essential skills students should develop as per CBSE competency-based education framework
4. Relevant CBSE educational standards and learning outcomes this lesson should align with
5. Prerequisites students should already understand from Class 9 or previous Class 10 chapters
6. Appropriate scope for a single lesson on this topic considering CBSE syllabus time allocation
7. Suggested sequence of concepts (from basic to advanced) following NCERT pedagogical approach
8. Potential misconceptions or difficulties CBSE Class 10 students might encounter with this topic, considering board exam preparation needs

Format your response as a structured JSON object with these categories, ensuring content is relevant to CBSE board examination patterns and Indian educational context.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self._call_llm(messages)
        
        # Parse the JSON response
        try:
            result = json.loads(response)
        except json.JSONDecodeError:
            # If the response is not valid JSON, try to extract JSON using a simple heuristic
            try:
                json_start = response.find('{')
                json_end = response.rfind('}') + 1
                if json_start >= 0 and json_end > json_start:
                    json_str = response[json_start:json_end]
                    result = json.loads(json_str)
                else:
                    # Create a structured result manually
                    logger.warning("Could not parse JSON from curriculum expert response")
                    result = {
                        "learning_objectives": ["Understand " + topic],
                        "key_concepts": [topic],
                        "essential_skills": ["Understanding of " + topic],
                        "standards_alignment": [],
                        "prerequisites": [],
                        "scope": "Single lesson on " + topic,
                        "sequence": [topic],
                        "potential_misconceptions": []
                    }
            except Exception as e:
                logger.error(f"Error extracting JSON from response: {e}")
                result = {
                    "learning_objectives": ["Understand " + topic],
                    "key_concepts": [topic],
                    "essential_skills": ["Understanding of " + topic],
                    "standards_alignment": [],
                    "prerequisites": [],
                    "scope": "Single lesson on " + topic,
                    "sequence": [topic],
                    "potential_misconceptions": []
                }
        
        return {
            "curriculum_analysis": result,
            "subject": subject,
            "subtopics":subtopics,
            "topic": topic,
            "grade_level": grade_level
        }

In [190]:
class ContentCreationAgent(Agent):
    """Agent specialized in creating educational content based on curriculum requirements."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.3):
        """Initialize the content creation agent."""
        super().__init__(model_name, temperature)
        self.name = "ContentCreator"
        self.system_prompt = """You are an expert educational content creator with specialized knowledge of CBSE Class 10 curriculum and NCERT textbook standards. Your role is to:

1. Create accurate, comprehensive explanations aligned with NCERT textbook content and CBSE syllabus
2. Develop clear examples to explain the topics if needed
3. Design effective practice problems and activities that prepare students for CBSE board examinations
4. Structure content following NCERT pedagogical approach and logical progression
5. Ensure appropriate depth and breadth of coverage as per CBSE Class 10 standards
6. Incorporate assessment elements aligned with CBSE marking schemes and question patterns

Focus on creating content that is factually accurate, culturally relevant, clear, engaging, and specifically designed for CBSE Class 10 board exam preparation while following NCERT guidelines.
CRITICAL: You MUST respond with ONLY valid JSON format. No additional text before or after the JSON.


"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to generate educational content."""
        curriculum_analysis = input_data.get("curriculum_analysis", {})
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        subtopics = input_data.get("subtopics", "")
        #"subtopics":subtopics
        grade_level = input_data.get("grade_level", "")
        context = input_data.get("context", "")
        
        # Extract key information from curriculum analysis
        learning_objectives = curriculum_analysis.get("learning_objectives", [])
        key_concepts = curriculum_analysis.get("key_concepts", [])
        sequence = curriculum_analysis.get("sequence", [])
        
        # Format the learning objectives and key concepts as bullet points
        objectives_text = "\n".join([f"- {obj}" for obj in learning_objectives])
        concepts_text = "\n".join([f"- {concept}" for concept in key_concepts])
        sequence_text = "\n".join([f"{i+1}. {step}" for i, step in enumerate(sequence)])
        
        prompt = f"""
Create comprehensive educational content for a lesson on {topic} and sub-topics : {subtopics} in {subject} for {grade_level} students.

Learning Objectives:
{objectives_text}

Key Concepts to Cover:
{concepts_text}

Suggested Sequence:
{sequence_text}

Context from curriculum materials:
{context}

Return your response as a JSON object with exactly these keys:

{{
    "introduction": "Mentioning objectives and basic definitions around the {topic} and {subtopics}",
    "core_content": "Detailed explanations of each key concept and sub-topic following NCERT approach, with clear definitions, step-by-step explanations, Indian context examples, and numerical problems with answers",
    "examples": "2-3 detailed worked examples that demonstrate application of concepts, follow CBSE board exam question patterns, and include step-by-step solutions",
    "practice_activities": "Practice problems and activities that reinforce key learning objectives",
    "summary": "Summary that highlights important formulas/concepts for board exams and connects to upcoming topics"
}}

Ensure all content is specifically aligned with CBSE Class 10 standards.
"""

        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self._call_llm(messages)
        
        # Parse the JSON response
        try:
            result = json.loads(response)
        except json.JSONDecodeError:
            # If the response is not valid JSON, try to extract JSON using a simple heuristic
            try:
                json_start = response.find('{')
                json_end = response.rfind('}') + 1
                if json_start >= 0 and json_end > json_start:
                    json_str = response[json_start:json_end]
                    result = json.loads(json_str)
                else:
                    # Create a structured result manually
                    logger.warning("Could not parse JSON from content creator response")
                    result = {
                        "introduction": f"Introduction to {topic}",
                        "core_content": f"Explanation of {topic}",
                        "examples": f"Examples of {topic}",
                        "practice_activities": f"Practice activities for {topic}",
                        "summary": f"Summary of {topic}"
                    }
            except Exception as e:
                logger.error(f"Error extracting JSON from response: {e}")
                result = {
                    "introduction": f"Introduction to {topic}",
                    "core_content": f"Explanation of {topic}",
                    "examples": f"Examples of {topic}",
                    "practice_activities": f"Practice activities for {topic}",
                    "summary": f"Summary of {topic}"
                }

                
        
        # Create lesson components from the content
        components = []
        component_order = {
            "introduction": 1,
            "core_content": 2,
            "examples": 3,
            "practice_activities": 4,
            "summary": 5
        }
        
        for component_type, content in result.items():
            order = component_order.get(component_type, 99)
            component = LessonComponent(
                component_type=component_type.replace("_", " ").title(),
                content=content,
                order=order
            )
            components.append(component)
        
        return {
            "components": components,
            "subject": subject,
            "topic": topic,
            "subtopics":subtopics,
            "grade_level": grade_level
        }

In [191]:
class PedagogyExpertAgent(Agent):
    """Agent specialized in pedagogical approaches and teaching strategies."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.2):
        """Initialize the pedagogy expert agent."""
        super().__init__(model_name, temperature)
        self.name = "PedagogyExpert"
        self.system_prompt = """You are an expert in pedagogy and teaching methodologies with deep knowledge of how students learn. Your role is to:

1. Enhance lesson content with effective teaching strategies
2. Adapt content for different learning styles (visual, auditory, kinesthetic)
3. Incorporate engagement techniques to maintain student interest
4. Suggest differentiation approaches for diverse learners
5. Recommend assessment strategies aligned with learning objectives

Focus on making the lesson more effective, engaging, and accessible to all students.
"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to enhance content with pedagogical strategies."""
        components = input_data.get("components", [])
        curriculum_analysis = input_data.get("curriculum_analysis", {})
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        subtopics = input_data.get("subtopics", "")
        grade_level = input_data.get("grade_level", "")
        
        # Extract learning objectives
        learning_objectives = curriculum_analysis.get("learning_objectives", [])
        objectives_text = "\n".join([f"- {obj}" for obj in learning_objectives])
        
        # Process each component to enhance with pedagogical strategies
        enhanced_components = []
        
        for component in components:
            component_type = component.component_type
            content = component.content
            
            prompt = f"""
Enhance the following {component_type.lower()} for a lesson on {topic} in {subject} for {grade_level} students with effective pedagogical strategies.

Learning Objectives:
{objectives_text}

Original Content:
{content}

Please enhance this content by:
1. Incorporating diverse teaching strategies appropriate for this component
2. Adding elements that address different learning styles (visual, auditory, kinesthetic)
3. Including engagement techniques to maintain student interest
4. Suggesting differentiation approaches for diverse learners
5. Ensuring clear connections to the learning objectives

Provide the enhanced content in a well-structured format that a teacher could use directly.
"""
            
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ]
            
            enhanced_content = self._call_llm(messages)
            
            # Create enhanced component
            enhanced_component = LessonComponent(
                component_type=component.component_type,
                content=enhanced_content,
                order=component.order,
                metadata=component.metadata
            )
            enhanced_components.append(enhanced_component)
        
        # Add a new component for teaching strategies
        teaching_strategies_prompt = f"""
Create a "Teaching Strategies" section for a lesson on {topic} sub topic {subtopics}  in {subject} for {grade_level} students.

Learning Objectives:
{objectives_text}

This section should include:
1. Specific teaching strategies recommended for this lesson
2. Differentiation approaches for diverse learners (advanced, struggling, ELL, etc.)
3. Classroom management suggestions
4. Transition ideas between lesson components
5. Tips for addressing common misconceptions or difficulties

Provide practical, specific strategies that a teacher could implement directly.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": teaching_strategies_prompt}
        ]
        
        teaching_strategies = self._call_llm(messages)
        
        # Create teaching strategies component
        strategies_component = LessonComponent(
            component_type="Teaching Strategies",
            content=teaching_strategies,
            order=6,
            metadata={}
        )
        enhanced_components.append(strategies_component)
        
        # Add an assessment component
        assessment_prompt = f"""
Create an "Assessment" section for a lesson on {topic} and for sub topic {subtopics} in {subject} for {grade_level} students.

Learning Objectives:
{objectives_text}

This section should include:
1. Formative assessment strategies to use during the lesson
2. A summative assessment aligned with the learning objectives
3. Rubric or scoring criteria for evaluating student understanding
4. Extension or remediation suggestions based on assessment results

Provide practical, specific assessment approaches that align with the learning objectives.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": assessment_prompt}
        ]
        
        assessment_content = self._call_llm(messages)
        
        # Create assessment component
        assessment_component = LessonComponent(
            component_type="Assessment",
            content=assessment_content,
            order=7,
            metadata={}
        )
        enhanced_components.append(assessment_component)
        
        return {
            "enhanced_components": enhanced_components,
            "subject": subject,
            "topic": topic,
            "grade_level": grade_level,
            "subtopics": subtopics
        }



In [192]:
class ProblemSolvingExpertAgent(Agent):
    """Agent specialized in problem-solving techniques and methodologies."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.2):
        """Initialize the problem-solving expert agent."""
        super().__init__(model_name, temperature)
        self.name = "ProblemSolvingExpert"
        self.system_prompt = """You are an expert in problem-solving methodologies and critical thinking approaches with deep knowledge of how students develop analytical skills. Your role is to:

1. Design structured problem-solving frameworks for specific topics
2. Create step-by-step problem-solving strategies tailored to the subject matter
3. Develop critical thinking exercises that build analytical capabilities
4. Provide real-world problem scenarios that connect to the curriculum
5. Suggest scaffolding techniques to guide students through complex problem-solving processes

Focus on developing students' analytical thinking, logical reasoning, and systematic approach to solving problems in the given subject area.
"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to enhance content with problem-solving techniques."""
        components = input_data.get("components", [])
        curriculum_analysis = input_data.get("curriculum_analysis", {})
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        subtopics = input_data.get("subtopics", "")
        grade_level = input_data.get("grade_level", "")
        
        # Extract learning objectives
        learning_objectives = curriculum_analysis.get("learning_objectives", [])
        objectives_text = "\n".join([f"- {obj}" for obj in learning_objectives])
        
        # Process each component to enhance with problem-solving techniques
        enhanced_components = []
        
        for component in components:
            component_type = component.component_type
            content = component.content
            
            prompt = f"""
Transform the following {component_type.lower()} for a lesson on {topic} (subtopic: {subtopics}) in {subject} for {grade_level} students into a problem-solving focused approach.

Learning Objectives:
{objectives_text}

Original Content:
{content}

Please redesign this content by:
1. Incorporating specific problem-solving frameworks (e.g., IDEAL method, 5-step problem solving, design thinking)
2. Creating structured analytical thinking exercises related to the topic
3. Developing real-world problem scenarios that students can solve using the concepts
4. Including step-by-step problem-solving processes with clear reasoning steps
5. Adding critical thinking questions that guide students through logical analysis

Provide content that actively engages students in solving problems rather than passively receiving information.
"""
            
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ]
            
            enhanced_content = self._call_llm(messages)
            
            # Create enhanced component
            enhanced_component = LessonComponent(
                component_type=component.component_type,
                content=enhanced_content,
                order=component.order,
                metadata=component.metadata
            )
            enhanced_components.append(enhanced_component)
        
        # Add a new component for problem-solving strategies
        problem_solving_strategies_prompt = f"""
Create a "Problem-Solving Techniques" section for the topic "{topic}" and subtopic "{subtopics}" in {subject} for {grade_level} students.

Learning Objectives:
{objectives_text}

This section should include:
1. Specific problem-solving frameworks applicable to this topic (e.g., scientific method, mathematical problem-solving steps, historical analysis framework)
2. Step-by-step analytical thinking processes students should follow
3. Common problem types in this subject area and systematic approaches to solve them
4. Critical thinking questions and prompts that guide logical reasoning
5. Troubleshooting strategies for when students get stuck
6. Real-world problem scenarios that require application of the topic concepts

Provide practical, systematic approaches that students can apply independently to solve problems in this subject area.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": problem_solving_strategies_prompt}
        ]
        
        problem_solving_strategies = self._call_llm(messages)
        
        # Create problem-solving strategies component
        strategies_component = LessonComponent(
            component_type="Problem-Solving Techniques",
            content=problem_solving_strategies,
            order=6,
            metadata={}
        )
        enhanced_components.append(strategies_component)
        
        # Add a problem-based assessment component
        assessment_prompt = f"""
Create a "Problem-Based Assessment" section for the topic "{topic}" and subtopic "{subtopics}" in {subject} for {grade_level} students.

Learning Objectives:
{objectives_text}

This section should include:
1. Complex, multi-step problems that require application of the learned concepts
2. Real-world scenarios where students must identify the problem, analyze it, and propose solutions
3. Rubric for evaluating problem-solving process (not just final answers)
4. Self-reflection questions for students to assess their own problem-solving approach
5. Progressive problem sets that build from simple to complex analytical challenges
6. Collaborative problem-solving activities that encourage peer discussion and reasoning

Focus on assessing students' analytical thinking process, logical reasoning, and systematic approach to problem-solving rather than memorization.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": assessment_prompt}
        ]
        
        assessment_content = self._call_llm(messages)
        
        # Create assessment component
        assessment_component = LessonComponent(
            component_type="Problem-Based Assessment",
            content=assessment_content,
            order=7,
            metadata={}
        )
        enhanced_components.append(assessment_component)
        
        return {
            "enhanced_components": enhanced_components,
            "subject": subject,
            "topic": topic,
            "grade_level": grade_level,
            "subtopics": subtopics
        }


In [193]:
class QualityAssuranceAgent(Agent):
    """Agent specialized in evaluating and providing feedback on lesson quality."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.1):
        """Initialize the quality assurance agent."""
        super().__init__(model_name, temperature)
        self.name = "QualityAssurance"
        self.system_prompt = """You are an expert educational quality assurance specialist with deep knowledge of curriculum standards, pedagogy, and content accuracy. Your role is to:

1. Evaluate lesson content against quality criteria
2. Identify strengths and areas for improvement
3. Check alignment with learning objectives and standards
4. Verify factual accuracy and pedagogical soundness
5. Provide specific, actionable feedback for improvement

Be thorough, specific, and constructive in your evaluation.
"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to evaluate lesson quality and provide feedback."""
        components = input_data.get("enhanced_components", [])
        curriculum_analysis = input_data.get("curriculum_analysis", {})
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        grade_level = input_data.get("grade_level", "")
        
        # Extract learning objectives
        learning_objectives = curriculum_analysis.get("learning_objectives", [])
        objectives_text = "\n".join([f"- {obj}" for obj in learning_objectives])
        
        # Combine all components into a single lesson text for evaluation
        lesson_text = ""
        for component in sorted(components, key=lambda x: x.order):
            lesson_text += f"## {component.component_type}\n\n{component.content}\n\n"
        
        prompt = f"""
Evaluate the following lesson on {topic} in {subject} for {grade_level} students against quality criteria.

Learning Objectives:
{objectives_text}

Lesson Content:
{lesson_text}

Please evaluate this lesson on the following criteria:
1. Alignment with learning objectives
2. Content accuracy and completeness
3. Pedagogical effectiveness
4. Engagement and student-centeredness
5. Differentiation and inclusivity
6. Assessment alignment
7. Overall structure and flow

For each criterion:
- Provide a score from 1-5 (where 5 is excellent)
- Identify specific strengths
- Identify specific areas for improvement
- Provide actionable recommendations

Also provide an overall evaluation summary and a final quality score from 1-10.

Format your response as a JSON object with these categories.
"""
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self._call_llm(messages)
        
        # Parse the JSON response
        try:
            result = json.loads(response)
        except json.JSONDecodeError:
            # If the response is not valid JSON, try to extract JSON using a simple heuristic
            try:
                json_start = response.find('{')
                json_end = response.rfind('}') + 1
                if json_start >= 0 and json_end > json_start:
                    json_str = response[json_start:json_end]
                    result = json.loads(json_str)
                else:
                    # Create a structured result manually
                    logger.warning("Could not parse JSON from quality assurance response")
                    result = {
                        "alignment": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "content_accuracy": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "pedagogical_effectiveness": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "engagement": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "differentiation": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "assessment_alignment": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "structure_and_flow": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                        "overall_summary": f"Basic evaluation of {topic} lesson",
                        "quality_score": 6
                    }
            except Exception as e:
                logger.error(f"Error extracting JSON from response: {e}")
                result = {
                    "alignment": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "content_accuracy": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "pedagogical_effectiveness": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "engagement": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "differentiation": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "assessment_alignment": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "structure_and_flow": {"score": 3, "strengths": [], "areas_for_improvement": [], "recommendations": []},
                    "overall_summary": f"Basic evaluation of {topic} lesson",
                    "quality_score": 6
                }
        
        # Extract feedback for each component
        feedback = []
        for category, evaluation in result.items():
            if isinstance(evaluation, dict) and "recommendations" in evaluation:
                for recommendation in evaluation["recommendations"]:
                    feedback.append(f"{category.replace('_', ' ').title()}: {recommendation}")
        
        # Add overall summary
        if "overall_summary" in result:
            feedback.append(f"Overall: {result['overall_summary']}")
        
        # Extract quality score
        quality_score = result.get("quality_score", 6)
        
        return {
            "evaluation": result,
            "feedback": feedback,
            "quality_score": quality_score,
            "components": components,
            "subject": subject,
            "topic": topic,
            "grade_level": grade_level
        }



In [194]:
class RefinementAgent(Agent):
    """Agent specialized in refining and improving lesson content based on feedback."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = 0.3):
        """Initialize the refinement agent."""
        super().__init__(model_name, temperature)
        self.name = "Refinement"
        self.system_prompt = """You are an expert educational content refiner with deep knowledge of curriculum, pedagogy, and effective teaching. Your role is to:

1. Improve lesson content based on specific feedback
2. Enhance explanations, examples, and activities
3. Address identified gaps or weaknesses
4. Ensure alignment with learning objectives
5. Polish language and presentation for clarity and engagement

Focus on making targeted, substantive improvements while maintaining the original structure and purpose.
"""
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process input data to refine lesson content based on feedback."""
        components = input_data.get("components", [])
        feedback = input_data.get("feedback", [])
        evaluation = input_data.get("evaluation", {})
        curriculum_analysis = input_data.get("curriculum_analysis", {})
        subject = input_data.get("subject", "")
        topic = input_data.get("topic", "")
        grade_level = input_data.get("grade_level", "")
        
        # Extract learning objectives
        learning_objectives = curriculum_analysis.get("learning_objectives", [])
        objectives_text = "\n".join([f"- {obj}" for obj in learning_objectives])
        
        # Format feedback as bullet points
        feedback_text = "\n".join([f"- {item}" for item in feedback])
        
        # Process each component to refine based on feedback
        refined_components = []
        
        for component in components:
            component_type = component.component_type
            content = component.content
            
            # Find specific feedback for this component type
            component_feedback = []
            for item in feedback:
                if component_type.lower() in item.lower():
                    component_feedback.append(item)
            
            component_feedback_text = "\n".join([f"- {item}" for item in component_feedback])
            if not component_feedback_text:
                component_feedback_text = "No specific feedback for this component."
            
            prompt = f"""
Refine the following {component_type.lower()} for a lesson on {topic} in {subject} for {grade_level} students based on quality feedback.

Learning Objectives:
{objectives_text}

Original Content:
{content}

Feedback (overall):
{feedback_text}

Specific Feedback for this component:
{component_feedback_text}

Please refine this content by:
1. Addressing the specific feedback provided
2. Enhancing explanations, examples, or activities as needed
3. Ensuring strong alignment with the learning objectives
4. Improving clarity, engagement, and effectiveness
5. Maintaining the original purpose and structure while making substantive improvements

Provide the refined content in a well-structured format that a teacher could use directly.
"""
            
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ]
            
            refined_content = self._call_llm(messages)
            
            # Create refined component
            refined_component = LessonComponent(
                component_type=component.component_type,
                content=refined_content,
                order=component.order,
                metadata=component.metadata
            )
            refined_components.append(refined_component)
        
        return {
            "refined_components": refined_components,
            "subject": subject,
            "topic": topic,
            "grade_level": grade_level,
            "feedback": feedback
        }



In [195]:

class OrchestratorAgent:
    """Agent that orchestrates the entire lesson generation workflow."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, embedding_model: str = EMBEDDING_MODEL):
        """Initialize the orchestrator with all specialized agents."""
        self.curriculum_expert = CurriculumExpertAgent(model_name)
        self.content_creator = ContentCreationAgent(model_name)
        self.pedagogy_expert = PedagogyExpertAgent(model_name)
        self.Problem_Solving_Expert = ProblemSolvingExpertAgent(model_name)
        self.quality_assurance = QualityAssuranceAgent(model_name)
        self.refinement_agent = RefinementAgent(model_name)
        
        # Initialize embedding model
        self.embeddings = OllamaEmbeddings(
            model=embedding_model,
            base_url=OLLAMA_BASE_URL
        )
        
        # For tracking progress and errors
        self.errors = []
        self.progress = []
    
    def _log_progress(self, stage: str, message: str):
        """Log progress of the workflow."""
        self.progress.append({"stage": stage, "message": message})
        logger.info(f"[{stage}] {message}")
    
    def _log_error(self, stage: str, error: str):
        """Log errors encountered during the workflow."""
        self.errors.append({"stage": stage, "error": error})
        logger.error(f"[{stage}] {error}")
    
    def _extract_text_from_pdf(self, pdf_path: str) -> List[Document]:
        """Extract text from a PDF file and create Document objects."""
        self._log_progress("PDF Processing", f"Extracting text from {pdf_path}")
        
        try:
            reader = PdfReader(pdf_path)
            documents = []
            
            # Process each page as a separate document
            for i, page in enumerate(reader.pages):
                text = page.extract_text()
                
                if text.strip():  # Only add non-empty pages
                    documents.append(
                        Document(
                            page_content=text.strip(),
                            metadata={
                                "page": i + 1,
                                "source": os.path.basename(pdf_path)
                            }
                        )
                    )
            
            self._log_progress("PDF Processing", f"Created {len(documents)} documents from PDF")
            return documents
        
        except Exception as e:
            self._log_error("PDF Processing", f"Error extracting text from PDF: {str(e)}")
            return []
    
    def _create_vector_store(self, documents: List[Document], persist_directory: str) -> Chroma:
        """Create a vector store from documents."""
        self._log_progress("Vector Store", f"Creating vector store with {len(documents)} documents")
        
        try:
            # Split documents into smaller chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=500,
                chunk_overlap=50,
                separators=["\n\n", "\n", ". ", " ", ""]
            )
            
            splits = text_splitter.split_documents(documents)
            self._log_progress("Vector Store", f"Split {len(documents)} documents into {len(splits)} chunks")
            
            # Create vector store from documents
            vectorstore = Chroma.from_documents(
                documents=splits,
                embedding=self.embeddings,
                persist_directory=persist_directory
            )
            
            # Persist the vector store to disk
            vectorstore.persist()
            self._log_progress("Vector Store", f"Vector store created and saved to {persist_directory}")
            
            return vectorstore
        
        except Exception as e:
            self._log_error("Vector Store", f"Error creating vector store: {str(e)}")
            return None
    
    def _get_or_create_vector_store(self, pdf_path: str) -> Tuple[Chroma, bool]:
        """Get existing vector store or create a new one if it doesn't exist."""
        # Create a document identifier from the PDF filename
        document_id = os.path.basename(pdf_path).replace('.pdf', '')
        
        # Calculate a hash of the PDF file to detect changes
        file_hash = hashlib.md5(open(pdf_path, 'rb').read()).hexdigest()[:10]
        
        # Define the persistent directory with document ID and hash
        persist_directory = f"chroma_db_{document_id}_{file_hash}"
        
        # Check if this exact vector store already exists
        if os.path.exists(persist_directory) and os.listdir(persist_directory):
            self._log_progress("Vector Store", f"Vector store for {document_id} already exists at {persist_directory}")
            
            # Load the existing vector store
            vectorstore = Chroma(
                persist_directory=persist_directory,
                embedding_function=self.embeddings
            )
            self._log_progress("Vector Store", f"Loaded existing vector store with {vectorstore._collection.count()} documents")
            
            return vectorstore, False
        else:
            # Extract text from PDF
            documents = self._extract_text_from_pdf(pdf_path)
            
            # Create vector store from documents
            vectorstore = self._create_vector_store(documents, persist_directory)
            
            return vectorstore, True
    
    def _retrieve_context(self, vectorstore: Chroma, query: str, k: int = 5) -> str:
        """Retrieve relevant context from the vector store."""
        self._log_progress("Context Retrieval", f"Retrieving context for query: {query}")
        
        try:
            docs = vectorstore.similarity_search(query, k=k)
            context = "\n\n".join([doc.page_content for doc in docs])
            self._log_progress("Context Retrieval", f"Retrieved {len(docs)} documents for context")
            
            return context
        
        except Exception as e:
            self._log_error("Context Retrieval", f"Error retrieving context: {str(e)}")
            return ""
    
    def generate_lesson(self, 
                        subject: str, 
                        topic: str, 
                        grade_level: str, 
                        pdf_path: str = None,
                        subtopics: str = "",
                        special_requirements: str = "") -> Lesson:
        """Generate a complete lesson using the multi-agent workflow."""
        self._log_progress("Initialization", f"Starting lesson generation for {subject}: {topic}")
        
        # Step 1: Initialize vector store from PDF if provided
        vectorstore = None
        if pdf_path and os.path.exists(pdf_path):
            vectorstore, is_new = self._get_or_create_vector_store(pdf_path)
            if not vectorstore:
                self._log_error("Initialization", "Failed to create or load vector store")
                return None
        
        # Step 2: Retrieve relevant context
        context = ""
        if vectorstore:
            search_query = f"{subject} {topic} {subtopics}".strip()
            context = self._retrieve_context(vectorstore, search_query, k=5)
            self._log_progress("RAG-documets-check", f"Output: {context}")
        
        # Step 3: Curriculum analysis
        self._log_progress("Curriculum Analysis", "Analyzing curriculum requirements")
        curriculum_input = {
            "subject": subject,
            "sub_topic": subtopics,
            "topic": topic,
            "grade_level": grade_level,
            "special_requirements": special_requirements,
            "context": context
        }
        curriculum_output = self.curriculum_expert.process(curriculum_input)
        self._log_progress("Curriculum Analysis", f"Output: {curriculum_output}")
        
        # Step 4: Content creation
        self._log_progress("Content Creation", "Generating initial lesson content")
        content_input = {
            **curriculum_output,
            "context": context
        }
        content_output = self.content_creator.process(content_input)
        self._log_progress("Content Creation", f"Output: {content_output}")
        
        # Step 5: Pedagogical enhancement
        self._log_progress("Problem_Solving_Expert", "Problem_Solving_Expert with teaching strategies")
        pedagogy_input = {
            **content_output,
            "curriculum_analysis": curriculum_output.get("curriculum_analysis", {})
        }


        pedagogy_output = self.Problem_Solving_Expert.process(pedagogy_input)
        self._log_progress("Pedagogical Enhancement", f"Output: {pedagogy_output}")



        
        # Step 6: Quality assurance
        self._log_progress("Quality Assurance", "Evaluating lesson quality")
        qa_input = {
            **pedagogy_output,
            "curriculum_analysis": curriculum_output.get("curriculum_analysis", {})
        }
        qa_output = self.quality_assurance.process(qa_input)
        self._log_progress("Quality Assurance", f"Output: {qa_output}")
        
        # Step 7: Refinement
        self._log_progress("Refinement", "Refining lesson based on feedback")
        refinement_input = {
            **qa_output,
            "curriculum_analysis": curriculum_output.get("curriculum_analysis", {})
        }
        refinement_output = self.refinement_agent.process(refinement_input)
        self._log_progress("Refinement", f"Output: {refinement_output}")
        
        # Step 8: Create final lesson
        self._log_progress("Finalization", "Creating final lesson")
        
        # Create lesson metadata
        curriculum_analysis = curriculum_output.get("curriculum_analysis", {})
        metadata = LessonMetadata(
            subject=subject,
            grade_level=grade_level,
            topic=topic,
            subtopics=subtopics.split(",") if subtopics else [],
            learning_objectives=curriculum_analysis.get("learning_objectives", []),
            standards_alignment=curriculum_analysis.get("standards_alignment", []),
            prerequisites=curriculum_analysis.get("prerequisites", []),
            target_skills=curriculum_analysis.get("essential_skills", [])
        )
        
        # Create lesson with refined components
        lesson = Lesson(
            title=f"{topic} - {subject} Lesson for {grade_level}",
            metadata=metadata,
            components=refinement_output.get("refined_components", []),
            quality_score=qa_output.get("quality_score"),
            feedback=qa_output.get("feedback", [])
        )
        
        self._log_progress("Completion", f"Lesson generation complete with quality score: {lesson.quality_score}/10")
        
        return lesson
    
    def generate_lesson_new(self, 
                    subject: str, 
                    topic: str, 
                    grade_level: str, 
                    pdf_path: str = None,
                    subtopics: str = "",
                    special_requirements: str = "") -> Lesson:
        """Generate a complete lesson using the multi-agent workflow."""
        self._log_progress("Initialization", f"Starting lesson generation for {subject}: {topic}")
        
        # Step 1: Initialize vector store from PDF if provided
        vectorstore = None
        if pdf_path and os.path.exists(pdf_path):
            vectorstore, is_new = self._get_or_create_vector_store(pdf_path)
            if not vectorstore:
                self._log_error("Initialization", "Failed to create or load vector store")
                return None
        
        # Step 2: Retrieve relevant context
        context = ""
        if vectorstore:
            search_query = f"{subject} {topic} {subtopics}".strip()
            context = self._retrieve_context(vectorstore, search_query, k=5)
        
        # Step 3: Curriculum analysis
        self._log_progress("Curriculum Analysis", "Analyzing curriculum requirements")
        curriculum_input = {
            "subject": subject,
            "sub_topic": subtopics,
            "topic": topic,
            "grade_level": grade_level,
            "special_requirements": special_requirements,
            "context": context
        }
        curriculum_output = self.curriculum_expert.process(curriculum_input)
        
        # Step 4: Content creation
        self._log_progress("Content Creation", "Generating initial lesson content")
        content_input = {
            **curriculum_output,
            "context": context
        }
        content_output = self.content_creator.process(content_input)
        
        # ❌ Step 5: Skipped Pedagogical Enhancement
        # Directly proceed to QA using content_output

        # Step 6: Quality assurance
        self._log_progress("Quality Assurance", "Evaluating lesson quality")
        qa_input = {
            **content_output,
            "curriculum_analysis": curriculum_output.get("curriculum_analysis", {})
        }
        qa_output = self.quality_assurance.process(qa_input)
        
        # Step 7: Refinement
        self._log_progress("Refinement", "Refining lesson based on feedback")
        refinement_input = {
            **qa_output,
            "curriculum_analysis": curriculum_output.get("curriculum_analysis", {})
        }
        refinement_output = self.refinement_agent.process(refinement_input)
        
        # Step 8: Create final lesson
        self._log_progress("Finalization", "Creating final lesson")
        
        curriculum_analysis = curriculum_output.get("curriculum_analysis", {})
        metadata = LessonMetadata(
            subject=subject,
            grade_level=grade_level,
            topic=topic,
            subtopics=subtopics.split(",") if subtopics else [],
            learning_objectives=curriculum_analysis.get("learning_objectives", []),
            standards_alignment=curriculum_analysis.get("standards_alignment", []),
            prerequisites=curriculum_analysis.get("prerequisites", []),
            target_skills=curriculum_analysis.get("essential_skills", [])
        )
        
        lesson = Lesson(
            title=f"{topic} - {subject} Lesson for {grade_level}",
            metadata=metadata,
            components=refinement_output.get("refined_components", []),
            quality_score=qa_output.get("quality_score"),
            feedback=qa_output.get("feedback", [])
        )
        
        self._log_progress("Completion", f"Lesson generation complete with quality score: {lesson.quality_score}/10")
        
        return lesson

        
    


In [196]:
def generate_lesson(subject: str, 
                   topic: str, 
                   grade_level: str, 
                   pdf_path: str = None,
                   subtopics: str = "",
                   special_requirements: str = "",
                   output_format: str = "markdown") -> str:
    """
    Generate a high-quality lesson using the multi-agent workflow.
    
    Args:
        subject: The subject of the lesson (e.g., "Physics", "Mathematics")
        topic: The main topic of the lesson (e.g., "Electricity", "Fractions")
        grade_level: The grade level for the lesson (e.g., "10th Grade", "Elementary")
        pdf_path: Optional path to a PDF with curriculum materials
        subtopics: Optional comma-separated list of subtopics to cover
        special_requirements: Optional special requirements or focus areas
        output_format: Format for the output ("markdown" or "json")
        
    Returns:
        The generated lesson in the specified format
    """
    # Initialize the orchestrator
    orchestrator = OrchestratorAgent()
    
    # Generate the lesson
    lesson = orchestrator.generate_lesson(
        subject=subject,
        topic=topic,
        grade_level=grade_level,
        pdf_path=pdf_path,
        subtopics=subtopics,
        special_requirements=special_requirements
    )
    
    # Return the lesson in the requested format
    if output_format.lower() == "json":
        return lesson.to_json()
    else:
        return lesson.to_markdown()


In [197]:
# # Example usage
# if __name__ == "__main__":
#     import argparse
    
#     parser = argparse.ArgumentParser(description="Generate a high-quality lesson using a multi-agent workflow")
#     parser.add_argument("--subject", required=True, help="Subject of the lesson")
#     parser.add_argument("--topic", required=True, help="Main topic of the lesson")
#     parser.add_argument("--grade", required=True, help="Grade level for the lesson")
#     parser.add_argument("--pdf", help="Path to a PDF with curriculum materials")
#     parser.add_argument("--subtopics", default="", help="Comma-separated list of subtopics to cover")
#     parser.add_argument("--requirements", default="", help="Special requirements or focus areas")
#     parser.add_argument("--format", default="markdown", choices=["markdown", "json"], help="Output format")
#     parser.add_argument("--output", help="Output file path (if not provided, prints to stdout)")
    
#     args = parser.parse_args()


- check why it isnt sending feedback again and again to refine
- prompt engg last 2 agents and ppt
- implement internet search also in this - another agents , search using key words

- put past year resources and other PDFs in RAG database




In [198]:
    # Generate the lesson
lesson_content = generate_lesson(
        subject="physics]",
        topic="electricty",
        grade_level="10th",
        pdf_path="/raid1/Aryan/Evaluation /ch11_10th_grade.pdf",
        subtopics="ohm's law",
        special_requirements="alot of solved examples",
        output_format="markdown"
    )

2025-05-29 00:20:40,549 - __main__ - INFO - [Initialization] Starting lesson generation for physics]: electricty
2025-05-29 00:20:40,560 - __main__ - INFO - [Vector Store] Vector store for ch11_10th_grade already exists at chroma_db_ch11_10th_grade_bac719da90
2025-05-29 00:20:40,589 - __main__ - INFO - [Vector Store] Loaded existing vector store with 109 documents
2025-05-29 00:20:40,604 - __main__ - INFO - [Context Retrieval] Retrieving context for query: physics] electricty ohm's law
2025-05-29 00:20:42,306 - __main__ - INFO - [Context Retrieval] Retrieved 5 documents for context
2025-05-29 00:20:42,307 - __main__ - INFO - [RAG-documets-check] Output: In 1827, a German physicist Georg Simon Ohm
(1787–1854) found out the relationship between the current
I, flowing in a metallic wire and the potential difference
across its terminals. The potential difference, V, across the
ends of a given metallic wire in an electric circuit is directly
proportional to the current flowing through it,  

In [199]:
print(lesson_content)

# electricty - physics] Lesson for 10th

## Lesson Metadata

- **Subject:** physics]
- **Grade Level:** 10th
- **Topic:** electricty
- **Subtopics:** ohm's law
- **Difficulty Level:** intermediate
- **Estimated Duration:** 45 minutes

### Learning Objectives


### Prerequisites

- {'id': 1, 'description': 'Understanding of basic electrical concepts from Class 9 or previous Class 10 chapters'}

## Introduction

Here is the refined introduction for the lesson on electricity in physics for 10th-grade students:

**Electricity: Problem-Solving and Analytical Thinking**

**Learning Objectives:**

* Apply the IDEAL method to analyze and solve real-world problems related to electricity
* Develop analytical skills and logical reasoning abilities essential for tackling complex problems

**IDEAL Method: A Step-by-Step Approach to Problem-Solving**

1. **Identify**: Clearly define the problem or question you want to address.
2. **Define**: Gather information, identify key factors, and clarify the 

In [201]:
    # Generate the lesson
lesson_content_json = generate_lesson(
        subject="physics]",
        topic="electricty",
        grade_level="10th",
        pdf_path="/raid1/Aryan/Evaluation /ch11_10th_grade.pdf",
        subtopics="ohm's law,joules law",
        special_requirements="alot of solved examples",
        output_format="json"
    )

2025-05-29 00:26:12,316 - __main__ - INFO - [Initialization] Starting lesson generation for physics]: electricty
2025-05-29 00:26:12,328 - __main__ - INFO - [Vector Store] Vector store for ch11_10th_grade already exists at chroma_db_ch11_10th_grade_bac719da90
2025-05-29 00:26:12,342 - __main__ - INFO - [Vector Store] Loaded existing vector store with 109 documents
2025-05-29 00:26:12,354 - __main__ - INFO - [Context Retrieval] Retrieving context for query: physics] electricty ohm's law,joules law
2025-05-29 00:26:12,403 - __main__ - INFO - [Context Retrieval] Retrieved 5 documents for context
2025-05-29 00:26:12,405 - __main__ - INFO - [RAG-documets-check] Output: Electricity 189
Applying Ohm’s law [Eq. (11.5)], we get
H = I2 Rt (11.21)
This is known as Joule’s law of heating. The
law implies that heat produced in a resistor is
(i) directly proportional to the square of current
for a given resistance, (ii) directly proportional to
resistance for a given current, and (iii) directly
prop

In [202]:
print(lesson_content_json)

{
  "title": "electricty - physics] Lesson for 10th",
  "metadata": {
    "subject": "physics]",
    "grade_level": "10th",
    "topic": "electricty",
    "subtopics": [
      "ohm's law",
      "joules law"
    ],
    "learning_objectives": [
      "Understand electricty"
    ],
    "standards_alignment": [],
    "difficulty_level": "intermediate",
    "estimated_duration": "45 minutes",
    "prerequisites": [],
    "target_skills": [
      "Understanding of electricty"
    ]
  },
  "components": [
    {
      "component_type": "Introduction",
      "content": "Here is the refined introduction for the lesson on electricity:\n\n**Lesson Title:** \"Harnessing Electricity: A Problem-Solving Approach\"\n\n**Learning Objectives:**\n\n* Apply the IDEAL method to analyze electric circuits\n* Use Ohm's law, Joule's law, and electric potential difference to solve problems\n* Analyze relationships between current, voltage, and resistance\n\n**Problem-Solving Framework:** IDEAL Method (Identify,

In [206]:
import json
from typing import Dict, Any, Union, List
from langchain_community.llms import Ollama
import re

# Your constants
DEFAULT_MODEL = "llama3"
FASTER_MODEL = "llama3"
EMBEDDING_MODEL = "nomic-embed-text"
DEFAULT_TEMPERATURE = 0.2
DEFAULT_MAX_TOKENS = 4000
OLLAMA_BASE_URL = "http://localhost:11434"

class PresentationAgent:
    """Agent specialized in creating presentations from lesson content."""
    
    def __init__(self, model_name: str = DEFAULT_MODEL, temperature: float = DEFAULT_TEMPERATURE):
        """Initialize the presentation agent."""
        self.llm = Ollama(
            model=model_name,
            temperature=temperature,
            base_url=OLLAMA_BASE_URL
        )
        self.system_prompt = """You are an expert presentation designer for educational content. Create slides in this exact format:

Slide 1: Title Here
- Bullet point 1
- Bullet point 2
- Bullet point 3

Slide 2: Another Title
- More bullet points
- Keep it simple

Always start each slide with "Slide X:" followed by the title. Use bullet points starting with "-"."""
        self.name = "PresentationAgent"
    
    def _call_llm(self, messages: List[Dict[str, str]]) -> str:
        """Call the language model with the given messages using the same pattern as your agents."""
        # Format the messages for Ollama
        formatted_prompt = ""
        for msg in messages:
            if msg["role"] == "system":
                formatted_prompt += f"<s>[INST] <<SYS>>\n{msg['content']}\n<</SYS>>\n\n"
            elif msg["role"] == "user":
                if formatted_prompt:
                    formatted_prompt += f"{msg['content']} [/INST]"
                else:
                    formatted_prompt += f"[INST] {msg['content']} [/INST]"
            elif msg["role"] == "assistant":
                formatted_prompt += f" {msg['content']} </s><s>"
        
        # Call the LLM
        response = self.llm.invoke(formatted_prompt)
        return response
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process lesson content JSON and create presentation."""
        
        # Parse JSON if it's a string
        if isinstance(input_data.get("lesson_content"), str):
            try:
                lesson_data = json.loads(input_data["lesson_content"])
                print("✅ Successfully parsed JSON string")
            except json.JSONDecodeError as e:
                print(f"❌ Error parsing JSON: {e}")
                return {"error": "Invalid JSON format"}
        else:
            lesson_data = input_data.get("lesson_content", {})
        
        # Extract data from the correct JSON structure
        metadata = lesson_data.get("metadata", {})
        subject = metadata.get("subject", "").replace("]", "")
        topic = metadata.get("topic", "")
        subtopics = metadata.get("subtopics", [])
        grade_level = metadata.get("grade_level", "")
        
        # Convert subtopics list to string
        if isinstance(subtopics, list):
            subtopics_str = ", ".join(subtopics)
        else:
            subtopics_str = str(subtopics)
        
        print(f"📋 Extracted data:")
        print(f"   Subject: {subject}")
        print(f"   Topic: {topic}")
        print(f"   Subtopics: {subtopics_str}")
        print(f"   Grade Level: {grade_level}")
        
        # Create user prompt
        user_prompt = f"""Create a presentation about {topic} ({subtopics_str}) for {grade_level} {subject} students.

Create exactly 8 slides:
1. Introduction slide
2. What is {topic}?
3. Key concepts and formulas
4. Solved example 1
5. Solved example 2
6. Practice problems
7. Summary
8. Questions

Use the exact format: "Slide X: Title" followed by bullet points starting with "-"."""

        # Call LLM using agent pattern
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        print("🔄 Generating presentation content using Ollama...")
        formatted_presentation = self._call_llm(messages)
        
        # Debug: Print what LLM returned
        print("🔍 LLM Response (first 500 chars):")
        print(formatted_presentation[:500])
        print("=" * 50)
        
        # Parse slides
        slides = self._parse_slides(formatted_presentation)
        
        # If no slides found, create fallback
        if len(slides) == 0:
            print("⚠️ No slides found, creating fallback presentation...")
            slides = self._create_fallback_slides(metadata)
        
        # Create presentation structure
        presentation = {
            "metadata": {
                "title": f"{topic.title()} - {subtopics_str.title()}",
                "subject": subject,
                "topic": topic,
                "subtopics": subtopics_str,
                "grade_level": grade_level,
                "total_slides": len(slides),
                "created_date": "2025-05-29",
                "model_used": DEFAULT_MODEL
            },
            "slides": slides,
            "presentation_notes": {
                "estimated_duration": "45-50 minutes",
                "materials_needed": ["Projector", "Whiteboard", "Calculator"],
                "teaching_tips": [
                    "Pause after each example for questions",
                    "Encourage student participation",
                    "Use real-world analogies"
                ]
            }
        }
        
        # Save to file
        filename = self._save_presentation(presentation, topic)
        
        return {
            "presentation": presentation,
            "filename": filename,
            "slides_created": len(slides)
        }
    
    def _parse_slides(self, formatted_text: str) -> List[Dict[str, Any]]:
        """Parse formatted presentation text into slides."""
        slides = []
        current_slide = None
        
        lines = formatted_text.strip().split('\n')
        
        for line in lines:
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
                
            # Look for slide headers
            slide_match = re.match(r'^Slide\s*(\d+)\s*[:]\s*(.+)$', line, re.IGNORECASE)
            if slide_match:
                # Save previous slide
                if current_slide:
                    slides.append(current_slide)
                
                slide_num = slide_match.group(1)
                title = slide_match.group(2).strip()
                
                current_slide = {
                    "slide_number": f"Slide {slide_num}",
                    "title": title,
                    "content": []
                }
                print(f"📄 Found slide: {slide_num} - {title}")
                
            # Look for bullet points
            elif line.startswith('-') and current_slide:
                bullet_content = line[1:].strip()
                if bullet_content:
                    current_slide["content"].append(bullet_content)
                    
            # Look for other bullet formats
            elif line.startswith('•') and current_slide:
                bullet_content = line[1:].strip()
                if bullet_content:
                    current_slide["content"].append(bullet_content)
                    
            # Handle numbered lists
            elif re.match(r'^\d+\.', line) and current_slide:
                bullet_content = re.sub(r'^\d+\.\s*', '', line).strip()
                if bullet_content:
                    current_slide["content"].append(bullet_content)
        
        # Add the last slide
        if current_slide:
            slides.append(current_slide)
        
        print(f"📊 Parsed {len(slides)} slides total")
        return slides
    
    def _create_fallback_slides(self, metadata: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Create fallback slides if LLM parsing fails."""
        
        topic = metadata.get("topic", "Topic")
        subtopics = metadata.get("subtopics", [])
        
        if isinstance(subtopics, list):
            subtopics_str = ", ".join(subtopics)
        else:
            subtopics_str = str(subtopics)
        
        fallback_slides = [
            {
                "slide_number": "Slide 1",
                "title": f"Introduction to {topic.title()}",
                "content": [
                    f"Welcome to today's lesson on {topic}",
                    f"Focus on {subtopics_str}",
                    "Learning objectives and goals",
                    "Real-world applications"
                ]
            },
            {
                "slide_number": "Slide 2",
                "title": "Key Concepts",
                "content": [
                    "Fundamental principles",
                    "Important definitions",
                    "Core formulas and equations",
                    "Relationships between variables"
                ]
            },
            {
                "slide_number": "Slide 3",
                "title": "Solved Example 1",
                "content": [
                    "Problem statement",
                    "Given information",
                    "Step-by-step solution",
                    "Final answer with units"
                ]
            },
            {
                "slide_number": "Slide 4",
                "title": "Solved Example 2",
                "content": [
                    "More complex problem",
                    "Multiple steps involved",
                    "Application of concepts",
                    "Verification of results"
                ]
            },
            {
                "slide_number": "Slide 5",
                "title": "Practice Problems",
                "content": [
                    "Problems for students to solve",
                    "Varying difficulty levels",
                    "Group work opportunities",
                    "Discussion and solutions"
                ]
            },
            {
                "slide_number": "Slide 6",
                "title": "Summary",
                "content": [
                    "Key concepts reviewed",
                    "Important formulas",
                    "Main takeaways",
                    "Next lesson preview"
                ]
            }
        ]
        
        return fallback_slides
    
    def _save_presentation(self, presentation: Dict[str, Any], topic: str) -> str:
        """Save presentation to JSON file."""
        topic_clean = topic.replace(' ', '_').lower() if topic else 'lesson'
        filename = f"{topic_clean}_presentation.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(presentation, f, indent=4, ensure_ascii=False)
        
        print(f"✅ Presentation saved as: {filename}")
        print(f"📊 Total slides created: {presentation['metadata']['total_slides']}")
        print(f"🎯 Topic: {presentation['metadata']['topic']} - {presentation['metadata']['subtopics']}")
        
        return filename

# Usage function
def create_presentation_from_json(lesson_content_json: Union[str, Dict[str, Any]], model_name: str = DEFAULT_MODEL):
    """Create presentation using the agent pattern."""
    
    # Initialize the presentation agent
    presentation_agent = PresentationAgent(model_name=model_name)
    
    # Process the lesson content
    result = presentation_agent.process({
        "lesson_content": lesson_content_json
    })
    
    return result

# Usage with your existing setup
presentation_result = create_presentation_from_json(lesson_content_json, DEFAULT_MODEL)


✅ Successfully parsed JSON string
📋 Extracted data:
   Subject: physics
   Topic: electricty
   Subtopics: ohm's law, joules law
   Grade Level: 10th
🔄 Generating presentation content using Ollama...
🔍 LLM Response (first 500 chars):
Here is the presentation on electricity for 10th physics students:

Slide 1: Introduction to Electricity
- Electricity is a fundamental part of our daily lives
- It powers our homes, schools, and communities
- Understanding electricity is crucial for innovation and progress

Slide 2: What is Electricity?
- Electricity is a form of energy that arises from the movement of charged particles (electrons)
- It can be generated, transmitted, and used to power devices and systems
- Electricity is a vita
📄 Found slide: 1 - Introduction to Electricity
📄 Found slide: 2 - What is Electricity?
📄 Found slide: 3 - Key Concepts and Formulas
📄 Found slide: 4 - Solved Example 1 - Ohm's Law
📄 Found slide: 5 - Solved Example 2 - Joule's Law
📄 Found slide: 6 - Practice Problem